In [89]:
import scipy as sp
import scipy.sparse as spsp
import numpy as np
from math import cos, tan, exp,log, sin
from scipy.sparse import dia_matrix as jds
import scipy.sparse.linalg as spl
from scipy.sparse.linalg import spsolve
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d, Axes3D
from sklearn.preprocessing import  MinMaxScaler
import seaborn as sns

Nous essayons de resoudre

n est le nombre mailles

In [159]:
class SolverVol1D:
    
    #le maillage est uniforme pour simplifier les calculs
    def __init__(self, f, u, a, b, n, lamda = lambda x: 1):
        self.f = f
        self.u = u
        self.n = n
        self.lamda = lamda
        self.a = a
        self.b = b
        self.h = 1 / n
        self.fvector = []
        #vecteur contenant les n + 1 points du maillage
        self.fmail = np.linspace(0, 1, n + 1)
        #vecteur contenant les solutions exactes
        self.solexacte = []
        for i in np.arange(1, self.n):
            self.solexacte.append(u(self.fmail[i]))
            
    #stockage de la matrice au format JDS
    def initMatrix(self):
        offsets = np.array([-1, 0, 1])
        data0, data1, data2 = [], [], []
        c = self.b*self.h
        for i in np.arange(1, self.n):
            lamdai = self.lamda(self.fmail[i])
            lamdai1 = self.lamda(self.fmail[i + 1])
            lamdai2 = self.lamda(self.fmail[i - 1])
            sigmai = (2 * lamdai * lamdai1) / (self.h * (lamdai + lamdai1))
            sigmai1 = (2 * lamdai * lamdai2) / (self.h * (lamdai + lamdai2))
            if self.a > 0:
                data1.append(-1 * sigmai)
                data2.append(-1 * (sigmai1 + self.a))
                if i== 1:
                    data0.append(self.a + sigmai1 + 2 * self.lamda(self.fmail[1]) / self.h)
                elif i== self.n -1:
                    data0.append(self.a + sigmai1 + 2 * self.lamda(self.fmail[1]) / self.h)
                else:
                    data0.append(sigmai + sigmai1 + self.a + c)
                
                #On va aussi stocker le vecteur contenant les n - 1 points de f
                if i == 1:
                    self.fvector.append(self.h * self.f(self.fmail[i]) + (self.a + 2 * self.lamda(self.fmail[1]) / self.h) * self.u(self.fmail[0]))
                elif i == self.n - 1:
                    self.fvector.append(self.h * self.f(self.fmail[i]) + (2 * self.lamda(self.fmail[self.n]) / self.h) * self.u(self.fmail[self.n]))
                else:
                    self.fvector.append(self.h * self.f(self.fmail[i]))
            else:
                data2.append(-1 * sigmai1)
                data1.append(-1 * sigmai + self.a)
                data0.append(sigmai + sigmai1 - self.a + c)
                if i== 1:
                    data0.append(self.a + sigmai1 + 2 * self.lamda(self.fmail[1]) / self.h)
                elif i== self.n -1:
                    data0.append(self.a + sigmai1 + 2 * self.lamda(self.fmail[1]) / self.h)
                else:
                    data0.append(sigmai + sigmai1 + self.a + c)
                
                #On va aussi stocker le vecteur contenant les n - 1 points de f
                if i == 1:
                    self.fvector.append(self.h * self.f(self.fmail[i]) + (self.a + 2 * self.lamda(self.fmail[1]) / self.h) * self.u(self.fmail[0]))
                elif i == self.n - 1:
                    self.fvector.append(self.h * self.f(self.fmail[i]) + (2 * self.lamda(self.fmail[self.n]) / self.h) * self.u(self.fmail[self.n]))
                else:
                    self.fvector.append(self.h * self.f(self.fmail[i]))
        data2 = data2[1:]
        data2.append(0)
        data1 = [0] + data1[:-1]
        data = [data2, data0, data1]
        self.jdsdata = jds((data, offsets), shape = (self.n - 1, self.n - 1))
        return self.jdsdata
    
    #resolution du systeme d'equation
    def solve(self):
        mat = self.jdsdata.tocsc()
        self.solapproche = spsolve(self.jdsdata, self.fvector)
        return self.solapproche
    
    

Tests du programme

In [160]:
from sklearn.linear_model import LinearRegression
def vitesseConvergence(f, u, n1, n2):
    taille = 20
    tabh = np.array([k for k in range(n1,n2 + taille, taille)])
    error = []
    X= []
    for elem in range(0, len(tabh)):
        solver = SolverVol1D(f, u, a, b, tabh[elem], lamda)
        x.initMatrix()
        obtain = x.solve()
        err = abs(obtain-x.solexacte)
        error.append(log(max(err)))
    X= np.array([log(max(1/tabh[i],1/tabk[i])) for i in range(0, len(tabh))])
    reg = LinearRegression()
    reg.fit(X.reshape(-1,1), np.array(error))
    plt.figure()
    plt.plot(X,error)
    return reg.coef_

In [161]:
"'cette première fonction de test c'est celle des Polynomes en dim2 qui renvoit true ou False si"
"l'erreur est plus petite que la tolerance"
def testFunctionSimple(f, u, a, b, n, tol, lamda = lambda x: 1):
    x = SolverVol1D(f, u, a, b, n, lamda)
    x.initMatrix()
    obtain = x.solve()
    return(max(abs((obtain-x.solexacte))/x.solexacte)<tol)

def testFunctionNulle(f, u, a, b, n, tol, lamda = lambda x: 1):
    x = SolverVol1D(f, u, a, b, n, lamda)
    x.initMatrix()
    obtain = x.solve()
    return(max(abs((obtain-x.solexacte)))<tol)

#Cette deuxième fonction c'est pour d'autres types de fonctions dont on connait la solution le test est réussi si p>=2
def testFunctionComplex(f, u, a, b, n, m, ordre, tol, lamda = lambda x: 1):
    vitConv = vitesseConvergence(f,u,n,m)
    return vitConv >= ordre - tol


Courbe de progression du log erreur

In [162]:
from matplotlib.pyplot import plot

X = []
Y = []
for i in range(50, 300, 20):
    x = SolverVol1D(lambda x: 1+x, lambda x: x, 1, 1, i)
    x.initMatrix()
    obtain = x.solve()
    
    Y.append((max(abs((obtain-x.solexacte)))))
    X.append(i)

plot(X,Y)

IndexError: index 51 is out of bounds for axis 0 with size 51

In [163]:
x = SolverVol1D(lambda x: 1+x, lambda x: x, 1, 1, 10)
mat = x.initMatrix()
print(mat)
x.solve()

IndexError: index 11 is out of bounds for axis 0 with size 11

In [157]:
x.solexacte

[0.09090909090909091,
 0.18181818181818182,
 0.2727272727272727,
 0.36363636363636365,
 0.4545454545454546,
 0.5454545454545454,
 0.6363636363636364,
 0.7272727272727273,
 0.8181818181818182,
 0.9090909090909092]

# Test des cas de test

## Fonction nulle

In [158]:
testFunctionNulle(lambda x:0, lambda x:0, 1, 1, 100, 10^-8)

False

## Fonction Polynôme

In [ ]:
testFunctionSimple(lambda x: 1+x, lambda x: x, 1, 1, 10, tol, lamda = lambda x: 1)

## Fonction Quelconque

In [ ]:
testFunctionComplex(f, u, a, b, n, m, ordre, tol, lamda = lambda x: 1)